# Engineering Predictive Alpha Factors

## Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_datareader.data as web
from pathlib import Path

In [2]:
idx = pd.IndexSlice

# Load the dataset

In [3]:
data_path = Path('stock_prices.h5')
data_path

WindowsPath('stock_prices.h5')

In [4]:
with pd.HDFStore(data_path) as store:
    data = store[list(store.keys())[0]].loc[idx[:, '2006':'2016'], :]
print(data)

                        open       high        low      close     volume
ticker date                                                             
A      2006-01-03  22.786839  22.909642  22.387796  22.855063  3796200.0
       2006-01-04  22.889175  23.080202  22.766372  22.916465  3001300.0
       2006-01-05  22.820951  23.516836  22.820951  23.516836  3458800.0
       2006-01-06  23.537304  23.735153  23.257585  23.639640  4396500.0
       2006-01-09  23.639640  23.741976  23.482724  23.571416  2920500.0
...                      ...        ...        ...        ...        ...
ZUMZ   2016-12-23  20.950000  21.500000  20.950000  21.350000   532292.0
       2016-12-27  21.200000  21.700000  21.200000  21.450000   308004.0
       2016-12-28  21.550000  21.749900  21.325000  21.450000   165827.0
       2016-12-29  21.550000  22.050000  21.400000  21.900000   322108.0
       2016-12-30  21.900000  22.190000  21.600000  21.850000   295429.0

[7531907 rows x 5 columns]


In [5]:
data.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 7531907 entries, ('A', Timestamp('2006-01-03 00:00:00')) to ('ZUMZ', Timestamp('2016-12-30 00:00:00'))
Data columns (total 5 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   open    7531907 non-null  float64
 1   high    7531907 non-null  float64
 2   low     7531907 non-null  float64
 3   close   7531907 non-null  float64
 4   volume  7531907 non-null  float64
dtypes: float64(5)
memory usage: 316.8+ MB


In [6]:
dv = data.close.mul(data.volume)
dv

ticker  date      
A       2006-01-03    8.676239e+07
        2006-01-04    6.877919e+07
        2006-01-05    8.134003e+07
        2006-01-06    1.039317e+08
        2006-01-09    6.884032e+07
                          ...     
ZUMZ    2016-12-23    1.136443e+07
        2016-12-27    6.606686e+06
        2016-12-28    3.556989e+06
        2016-12-29    7.054165e+06
        2016-12-30    6.455124e+06
Length: 7531907, dtype: float64

In [7]:
dv_rk = dv.groupby(level='date').rank(ascending=False)

In [8]:
dv_rk[:, '2006-01-09']

ticker
A        241.0
AAL      566.0
AAN     2193.0
AAON    2033.0
AAP      509.0
         ...  
ZIXI    1764.0
ZLC      857.0
ZMH      182.0
ZQK      998.0
ZUMZ     908.0
Length: 2269, dtype: float64

In [9]:
selected500Stocks = dv.groupby(level='date').rank(ascending=False).unstack('ticker').dropna(thresh=8*252, axis=1).mean().nsmallest(500)

In [10]:
selected500Stocks

ticker
AAPL       2.115204
GOOGL      6.726255
MSFT       8.593716
BAC        9.465511
XOM       10.465511
            ...    
IGT      571.181293
NUAN     573.919104
CYH      574.426869
RSG      574.600939
SM       576.002167
Length: 500, dtype: float64

In [11]:
universe_stocks = data.loc[list(selected500Stocks.index), :]

In [12]:
universe_stocks

open       high        low      close       volume
ticker date                                                               
AAPL   2006-01-03   9.295670   9.606417   9.285132   9.606417  201808600.0
       2006-01-04   9.655252   9.764489   9.574288   9.634690  154900900.0
       2006-01-05   9.616698   9.625694   9.477903   9.558867  112355600.0
       2006-01-06   9.669389   9.857019   9.580714   9.805613  176114400.0
       2006-01-09   9.860874   9.921276   9.733646   9.773485  168760200.0
...                      ...        ...        ...        ...          ...
SM     2016-12-23  35.072860  35.769342  34.913664  35.291754     918559.0
       2016-12-27  35.620096  36.132309  35.203898  35.441001     901361.0
       2016-12-28  35.669845  36.097684  34.674870  34.943513    1437387.0
       2016-12-29  34.684820  34.903714  33.908740  34.535574    1296394.0
       2016-12-30  34.505724  35.639996  34.162458  34.306729    1491282.0

[1366286 rows x 5 columns]

In [13]:
universe_stocks.index.unique('ticker')

Index(['AAPL', 'GOOGL', 'MSFT', 'BAC', 'XOM', 'C', 'JPM', 'GE', 'INTC', 'CSCO',
       ...
       'THC', 'WSM', 'CMS', 'BMRN', 'CPN', 'IGT', 'NUAN', 'CYH', 'RSG', 'SM'],
      dtype='object', name='ticker', length=500)

In [14]:
universe_stocks.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1366286 entries, ('AAPL', Timestamp('2006-01-03 00:00:00')) to ('SM', Timestamp('2016-12-30 00:00:00'))
Data columns (total 5 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   open    1366286 non-null  float64
 1   high    1366286 non-null  float64
 2   low     1366286 non-null  float64
 3   close   1366286 non-null  float64
 4   volume  1366286 non-null  float64
dtypes: float64(5)
memory usage: 58.1+ MB


# Compute daily returns and keep only inliers with values between -100% and +100% as a basic check against data error.

In [15]:
universe_stocks['daily_returns'] = universe_stocks.groupby('ticker').close.pct_change()
universe_stocks = universe_stocks[universe_stocks.daily_returns.between(-1, 1)]
universe_stocks.drop('daily_returns', axis=1)
universe_stocks

open       high        low      close       volume  \
ticker date                                                                  
AAPL   2006-01-04   9.655252   9.764489   9.574288   9.634690  154900900.0   
       2006-01-05   9.616698   9.625694   9.477903   9.558867  112355600.0   
       2006-01-06   9.669389   9.857019   9.580714   9.805613  176114400.0   
       2006-01-09   9.860874   9.921276   9.733646   9.773485  168760200.0   
       2006-01-10   9.799188  10.524006   9.745212  10.391637  569967300.0   
...                      ...        ...        ...        ...          ...   
SM     2016-12-23  35.072860  35.769342  34.913664  35.291754     918559.0   
       2016-12-27  35.620096  36.132309  35.203898  35.441001     901361.0   
       2016-12-28  35.669845  36.097684  34.674870  34.943513    1437387.0   
       2016-12-29  34.684820  34.903714  33.908740  34.535574    1296394.0   
       2016-12-30  34.505724  35.639996  34.162458  34.306729    1491282.0   

                   daily_returns  
ticker date                       
AAPL   2006-01-04       0.002943  
       2006-01-05      -0.007870  
       2006-01-06       0.025813  
       2006-01-09      -0.003277  
       2006-01-10       0.063248  
...                          ...  
SM     2016-12-23       0.000847  
       2016-12-27       0.004229  
       2016-12-28      -0.014037  
       2016-12-29      -0.011674  
       2016-12-30      -0.006626  

[1365784 rows x 6 columns]

# Historical returns 

Compute historical returns for various time ranges, such as 1, 3, 5, 10, and 21 trading days, as well as longer periods like 2, 3, 6, and 12 months.

In [16]:
T = [1, 2, 3, 4, 5, 10, 21, 42, 63, 126, 252]

In [17]:
for k in T:
    universe_stocks[f'returns_{k}'] = universe_stocks.groupby('ticker').close.pct_change(k)

C:\Users\mndour\AppData\Local\Temp\ipykernel_1120\1200539020.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  universe_stocks[f'returns_{k}'] = universe_stocks.groupby('ticker').close.pct_change(k)


In [18]:
universe_stocks

open       high        low      close       volume  \
ticker date                                                                  
AAPL   2006-01-04   9.655252   9.764489   9.574288   9.634690  154900900.0   
       2006-01-05   9.616698   9.625694   9.477903   9.558867  112355600.0   
       2006-01-06   9.669389   9.857019   9.580714   9.805613  176114400.0   
       2006-01-09   9.860874   9.921276   9.733646   9.773485  168760200.0   
       2006-01-10   9.799188  10.524006   9.745212  10.391637  569967300.0   
...                      ...        ...        ...        ...          ...   
SM     2016-12-23  35.072860  35.769342  34.913664  35.291754     918559.0   
       2016-12-27  35.620096  36.132309  35.203898  35.441001     901361.0   
       2016-12-28  35.669845  36.097684  34.674870  34.943513    1437387.0   
       2016-12-29  34.684820  34.903714  33.908740  34.535574    1296394.0   
       2016-12-30  34.505724  35.639996  34.162458  34.306729    1491282.0   

                   daily_returns  returns_1  returns_2  returns_3  returns_4  \
ticker date                                                                    
AAPL   2006-01-04       0.002943        NaN        NaN        NaN        NaN   
       2006-01-05      -0.007870  -0.007870        NaN        NaN        NaN   
       2006-01-06       0.025813   0.025813   0.017740        NaN        NaN   
       2006-01-09      -0.003277  -0.003277   0.022452   0.014406        NaN   
       2006-01-10       0.063248   0.063248   0.059764   0.087120   0.078565   
...                          ...        ...        ...        ...        ...   
SM     2016-12-23       0.000847   0.000847   0.010253  -0.006164  -0.018811   
       2016-12-27       0.004229   0.004229   0.005079   0.014526  -0.001961   
       2016-12-28      -0.014037  -0.014037  -0.009867  -0.009029   0.000285   
       2016-12-29      -0.011674  -0.011674  -0.025547  -0.021427  -0.020598   
       2016-12-30      -0.006626  -0.006626  -0.018223  -0.032004  -0.027911   

                   returns_5  returns_10  returns_21  returns_42  returns_63  \
ticker date                                                                    
AAPL   2006-01-04        NaN         NaN         NaN         NaN         NaN   
       2006-01-05        NaN         NaN         NaN         NaN         NaN   
       2006-01-06        NaN         NaN         NaN         NaN         NaN   
       2006-01-09        NaN         NaN         NaN         NaN         NaN   
       2006-01-10        NaN         NaN         NaN         NaN         NaN   
...                      ...         ...         ...         ...         ...   
SM     2016-12-23   0.001129   -0.031404   -0.017724   -0.014175    0.040842   
       2016-12-27  -0.014661   -0.029956    0.000843    0.030671    0.092304   
       2016-12-28  -0.015971   -0.059454    0.061027   -0.006787   -0.036891   
       2016-12-29  -0.011393   -0.021427    0.087747    0.003469   -0.119964   
       2016-12-30  -0.027088   -0.009480   -0.134972    0.025275   -0.105176   

                   returns_126  returns_252  
ticker date                                  
AAPL   2006-01-04          NaN          NaN  
       2006-01-05          NaN          NaN  
       2006-01-06          NaN          NaN  
       2006-01-09          NaN          NaN  
       2006-01-10          NaN          NaN  
...                        ...          ...  
SM     2016-12-23     0.409827     0.720425  
       2016-12-27     0.315519     0.874546  
       2016-12-28     0.265771     0.861909  
       2016-12-29     0.287134     0.910871  
       2016-12-30     0.229428     0.759174  

[1365784 rows x 17 columns]

# Technical Indicators 

In [19]:
import talib
from talib import RSI, BBANDS, MACD, ATR, ADOSC, OBV, NATR

### Bollinger Bands

In [20]:
def compute_bb(close):
    
    high, mid, low = BBANDS(close)
    
    return pd.DataFrame({'bb_high': high, 'bb_low': low}, index=close.index)


In [21]:
universe_stocks = universe_stocks.join(universe_stocks.groupby(level='ticker').close.apply(compute_bb))  

### MACD

In [22]:
def compute_macd(close):
    macd = MACD(close)[0]
    
    return (macd - np.mean(macd))/np.std(macd)
           

In [23]:
universe_stocks['macd'] = universe_stocks.groupby(level='ticker', group_keys=False).close.apply(lambda x: MACD(x)[0])

### RSI

In [24]:
universe_stocks['rsi'] = universe_stocks.groupby(level='ticker').close.apply(RSI)

###  Chaikin A/D Oscillator (ADOSC)

In [25]:
universe_stocks['adosc'] = ADOSC(universe_stocks['high'], universe_stocks['low'], universe_stocks['close'], 
     universe_stocks['volume'], fastperiod=3, slowperiod=10 )

### On Balance Volume (OBV) 

In [26]:
universe_stocks['obv'] = OBV(universe_stocks['close'], universe_stocks['volume'])

### Create volatility metrics such as the Normalized Average True Range (NATR).

In [27]:
universe_stocks['natr'] = universe_stocks.groupby('ticker', group_keys=False).apply(lambda x: NATR(x.high, x.low, x.close))

### Compute rolling factor betas using the five Fama-French risk factors for different rolling windows of 3 and 12 months 

In [28]:
factor_data = (web.DataReader('F-F_Research_Data_5_Factors_2x3_daily', 'famafrench', 
                              start=2005)[0].rename(columns={'Mkt-RF': 'MARKET'}))
factor_data.index.names = ['date']

In [29]:

ret = f'returns_{1}'

windows = [63, 252]
for window in windows:
    print(window)
    betas = []
    for ticker, df in universe_stocks.groupby('ticker', group_keys=False):
        model_data = df[[ret]].merge(factor_data, on='date').dropna()
        model_data[ret] -= model_data.RF

        rolling_ols = RollingOLS(endog=model_data[ret], 
                                 exog=sm.add_constant(model_data[factors]), window=window)
        factor_model = rolling_ols.fit(params_only=True).params.rename(columns={'const':'ALPHA'})
        result = factor_model.assign(ticker=ticker).set_index('ticker', append=True).swaplevel()
        betas.append(result)
    betas = pd.concat(betas).rename(columns=lambda x: f'{x}_{window}')
    universe_stocks = universe_stocks.join(betas)

63


NameError: name 'RollingOLS' is not defined

In [ ]:
universe_stocks

### Compute the outcome variable that we will aim to predict, namely the 1-day forward returns.

In [ ]:
universe_stocks['returns_1_fwd'] = universe_stocks.groupby(level='ticker')['returns_1'].shift(-1)

In [ ]:
universe_stocks

In [ ]:
universe_stocks.info()